In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from state import get_state, queue
import os
import traci

In [22]:
from dotenv import load_dotenv
load_dotenv()


sumo_bin = os.getenv("SUMO")
sumo_gui_bin = os.getenv("SUMO-GUI")
simulConfig = os.getenv("SIMUL-CONFIG")

print(sumoBinary2)

/Library/Frameworks/EclipseSUMO.framework/Versions/1.22.0/EclipseSUMO/bin/sumo-gui


In [23]:
import numpy as np

from tensorflow.keras import Sequential, layers
from tensorflow.random import set_seed
from collections import deque
from tensorflow import keras
from tensorflow import reduce_sum, reduce_mean, one_hot, GradientTape
from tensorflow.keras.losses import MeanSquaredError
import matplotlib.pyplot as plt
import tensorflow as tf

In [24]:
set_seed(42)  # extra code – ensures reproducibility on the CPU

input_shape = [48]  # == env.observation_space.shape
n_outputs = 4  # == env.action_space.n
#[((Phase(duration=30.0, state='GGrGrrGGrGrr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grGgrrgrGgrr', minDur=20.0, maxDur=20.0), Phase(duration=30.0, state='GrrGGrGrrGGr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grrgrGgrrgrG', minDur=20.0, maxDur=20.0)), [0, 2, 4, 6])]
model_action = Sequential([
    layers.Dense(32,activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_shape=input_shape),
    layers.Dense(32, activation=tf.keras.layers.LeakyReLU(alpha=0.01)),
    layers.Dense(n_outputs, activation= 'linear')])

/Users/pr/.pyenv/versions/3.10.6/envs/sumo_env/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/Users/pr/.pyenv/versions/3.10.6/envs/sumo_env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
def epsilon_greedy_policy(state, epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs)  # random action
    else:
        Q_values = model_action.predict(state[np.newaxis], verbose=0)[0]
        print(f"state : {state}")
        print(f"Q_values : {Q_values}")
        return Q_values.argmax()  # optimal action according to the DQN

In [26]:
def sample_experiences(batch_size):
    indices = np.random.randint(len(replay_buffer), size=batch_size)
    batch = [replay_buffer[index] for index in indices]
    states, actions, rewards, next_states = [
        np.array([experience[field_index] for experience in batch])
        for field_index in range(4)
    ]
    return states, actions, rewards, next_states

In [27]:
batch_size = 32
discount_factor = 0.5
optimizer = keras.optimizers.Adam(learning_rate=0.05)
loss_fn = MeanSquaredError()

def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states = experiences  # a changer
    next_Q_values = model_action.predict(next_states, verbose=0)
    max_next_Q_values = next_Q_values.max(axis=1)
    # runs = 1.0 - (dones | truncateds)  # episode is not done or truncated
    target_Q_values = rewards + discount_factor * max_next_Q_values
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = one_hot(actions, n_outputs)
    with GradientTape() as tape:
        all_Q_values = model_action(states)
        Q_values = reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = reduce_mean(loss_fn(target_Q_values, Q_values))

    grads = tape.gradient(loss, model_action.trainable_variables)
    optimizer.apply_gradients(zip(grads, model_action.trainable_variables))

In [ ]:
rewards = []
best_score = 0
reward = 0
total_reward = 0
list_values = []
weights = []
wait_times = []
replay_buffer = deque(maxlen=10000)


sumoCmd = [sumo_bin, "-c", simulConfig, "--start"]
sumoCmd = [sumo_bin, "-c", simulConfig, "--start", "--no-warnings"]

for episode in range(300):
    if traci.isLoaded():
        traci.close()
    traci.start(sumoCmd)
    lane_ids =  traci.lane.getIDList()
    # print(lane_ids[0])

    trafic_light_ids = traci.trafficlight.getIDList()

    # state = np.array(queue(lane_ids))
    state=np.array(get_state(lane_ids))
    action=-1
    # print(state)
    wait_times.append(0)
    for step in range(10000): ## TO CHANGED
        epsilon = max(1 - episode / 300, 0.01)

        if step%1000 == 0:
            #print(f"longeur du buffer :{len(replay_buffer)}")
            ########################################################################
            ##Calcul de la reward
            # next_state = np.array(queue(lane_ids))
            next_state = np.array(get_state(lane_ids))
            # reward = calculate_reward(values, reward, total_reward)[0]
            # reward=(np.sum(state)-np.sum(next_state))
            reward = np.sum(state[:24])- np.sum(next_state[:24])
            replay_buffer.append((state, action, reward, next_state))

            list_values.append(queue(lane_ids))
            #########################################################################
            state=next_state
            action = epsilon_greedy_policy(state, epsilon)
            #print("action", action)
            traci.trafficlight.setPhase(trafic_light_ids[0],2*action)

            if len(replay_buffer) >= batch_size*10:
                training_step(batch_size)
                # new_weights = model_action.get_weights()
                # weights.append(new_weights)
                #print(f"Episode {episode}: new weights = {new_weights}")
            # else:
            #     print(f"Episode {episode}: pas assez de données dans le replay buffer.")
            # print(values)
            # if list_values:
            #    # print(f'list values {list_values[-1]}')


        traci.simulationStep()
    print(f'episode : {episode}')
    traci.close()


 Retrying in 1 seconds


episode : 0
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 33 ACT 10 BUF 0)                    
 Retrying in 1 seconds


episode : 1
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 33 ACT 6 BUF 0)                     
 Retrying in 1 seconds


episode : 2
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 1ms, vehicles TOT 33 ACT 1 BUF 0)                     
 Retrying in 1 seconds


episode : 3
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 33 ACT 1 BUF 0)                     
 Retrying in 1 seconds


...

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-0.52831924 -0.6091237  -0.56422484 -0.61358124]
state : [0 0 0 0 6 6 0 0 0 0 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 6 0 0 0 1 4 4 3
 0 0 0 0 0 3 0 0 0 0 0]
Q_values : [1.5436958 1.895864  9.198918  6.5582767]
state : [0 0 0 0 0 6 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 4 0
 0 0 0 4 0 3 0 0 0 6 0]
Q_values : [3.4549236 2.6489887 4.1470404 6.7818203]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 4 0 3 0 0 0 5 0 5 0]
Q_values : [-0.20638724 -0.43803614 -0.13923872 -0.01581113]
episode : 104
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 33 ACT 0 BUF 0)                     
 Retrying in 1 seconds


state : [0 1 0 0 1 6 0 0 0 0 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 6 0 0 0 1 4 4 3
 0 0 0 0 1 3 0 0 0 1 0]
Q_values : [2.1101139 1.929333  5.3057785 6.7915783]
state : [0 1 0 0 1 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 4 0 0
 0 4 0 0 0 3 0 5 0 0 0]
Q_values : [2.4014106 1.957263  4.7650785 2.6221402]
state : [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 4 0 4 0 0 0 6 0 1 0]
Q_values : [1.262271   0.84957516 1.0054868  1.1039858 ]
episode : 105
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 1ms, vehicles TOT 33 ACT 0 BUF 0)                     
 Retrying in 1 seconds


state : [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 6 6 0 0 0 4 4 1 2
 0 0 0 0 1 0 1 0 1 0 0]
Q_values : [-3.8082247 -2.9937973 -0.9437382 -0.9382163]
state : [0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 1 0 0 0 4 4 4 5
 0 0 0 0 0 0 0 4 0 0 0]
Q_values : [-1.5320675 -0.9393469  3.5677257  0.9894194]
state : [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 3 0 0 0 0 0 0 0 0 0]
Q_values : [0.7526064  0.6169416  0.7599739  0.58148336]
episode : 106
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 33 ACT 0 BUF 0)                     
 Retrying in 1 seconds


state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-0.6555831  -0.7990419  -0.62642497 -0.6888532 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 3 3 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-1.4191774 -1.5648332 -1.5148026 -1.6480514]
state : [0 0 0 0 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 6 0 0 0 4 4 4 5
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-2.2528064 -1.082718   2.8288     3.0410867]
state : [0 0 0 0 6 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 1 4 1 3
 0 2 0 0 0 3 0 5 0 0 0]
Q_values : [1.5681565 1.4076008 7.751856  3.3052745]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 3 0 0 0 0 0 5 0]
Q_values : [-0.15254086 -0.40662983 -0.04949248  0.05044699]
episode : 107
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 33 ACT 0 BUF 0)                     
 Retrying in 1 seconds


state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 3 3 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-1.1102936 -1.1712786 -1.4390616 -1.3991232]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 6 7 4 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-3.2301195 -2.707098  -2.1200154 -1.4674449]
state : [0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 4 3
 0 0 0 2 0 3 0 3 0 6 0]
Q_values : [-0.9286485  -0.34684193  1.134308    1.3571911 ]
state : [0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0
 0 4 0 2 0 3 0 0 0 5 0]
Q_values : [1.90293   1.5692751 1.1683782 1.8741229]
state : [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 1 0 0 0 0 0 0]
Q_values : [0.12209246 0.07323611 0.79057705 0.24313724]
episode : 108
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 33 ACT 1 BUF 0)                     
 Retrying in 1 seconds


In [ ]:
rewards = []
best_score = 0
reward = 0
total_reward = 0
list_values = []
weights = []
wait_times = []
# replay_buffer = deque(maxlen=2000)


sumoCmd = [sumo_gui_bin, "-c", simulConfig, "--start"]

if traci.isLoaded():
    traci.close()
traci.start(sumoCmd)
lane_ids =  traci.lane.getIDList()
# print(lane_ids[0])
# for lane in lane_ids:
#     print(traci.lane.getLastStepVehicleNumber(lane))
# north_lane = traci.lane.getLastStepVehicleNumber("N_0")
# south_lane = traci.lane.getLastStepVehicleNumber("S_0")
# east_lane = traci.lane.getLastStepVehicleNumber("E_0")
# west_lane = traci.lane.getLastStepVehicleNumber("W_0")
trafic_light_ids = traci.trafficlight.getIDList()

state = np.array(get_state(lane_ids))
action=-1
# print(state)
wait_times.append(0)
for step in range(10000): ## TO CHANGED
    #epsilon = max(1 - episode / 10, 0.01)
    nom_du_feu= traci.trafficlight.getIDList()[0]

    if step%500 == 0:
        state=np.array(get_state(lane_ids))
        action = epsilon_greedy_policy(state,0)*2
        # action = np.random.randint(8)
        print("action", action)
        #print(traci.trafficlight.getAllProgramLogics(nom_du_feu))
        #print(traci.trafficlight.getAllProgramLogics(nom_du_feu)[0].phases[action])
        traci.trafficlight.setPhase(trafic_light_ids[0],action)
    traci.simulationStep()

traci.close()
